In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import warnings; warnings.simplefilter('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from tqdm import tqdm

In [10]:
stocks = pd.read_csv('../input/sp-500-stock-prices/SP 500 Stock Prices 2014-2017.csv')
stocks.shape

FileNotFoundError: [Errno 2] No such file or directory: '../input/sp-500-stock-prices/SP 500 Stock Prices 2014-2017.csv'

### Check Volumes and Dollar Volumes 

In [ ]:
stocks['dvolume'] = stocks['close'] * stocks['volume']

fig, ax = plt.subplots(2,sharex=True, figsize=(16,6))
stocks.groupby('date')['volume'].sum().plot(ax=ax[0])
ax[0].set_title('Total Volume of S&P 500 Stocks')
stocks.groupby('date')['dvolume'].sum().plot(ax=ax[1])
ax[1].set_title('Total Dollar Volume of S&P 500 Stocks');

### Volume distribution of popoular stocks 

In [ ]:
pop_stocks = stocks.groupby('symbol')['dvolume'].mean().sort_values(
    ascending=False).head(10).index.to_list()

fig, ax = plt.subplots(2,sharex=True,figsize=(12,10))
sns.boxplot(x='symbol', y='volume', data=stocks, order=pop_stocks, ax=ax[0])
ax[0].set_title('Volume Distribution of Popular Stocks')
sns.boxplot(x='symbol', y='dvolume', data=stocks, order=pop_stocks, ax=ax[1])
ax[1].set_title('Dollar Volume Distribution of Popular Stocks');

### Volatile Stocks vs. Popular Stocks 

In [ ]:
# daily return
ror0 = stocks.sort_values(['symbol','date']).drop(['open','high','low','volume','dvolume'], axis=1)
ror0['return'] = ror0['close'].pct_change()

# drop first row of each stock
ror = ror0[ror0['symbol']==ror0['symbol'].shift()]

In [ ]:
high_vola = ror.groupby('symbol')['return'].std().sort_values(
    ascending=False)[:10].index.to_list()

fig, ax= plt.subplots(1,2,sharex=True,figsize=(16,8))
plt.suptitle('Distribution of Daily Return')
sns.boxplot(y='symbol', x='return', data=ror, order=high_vola, orient='h', ax=ax[0])
ax[0].set_title('Volatile Stocks')
sns.boxplot(y='symbol', x='return', data=ror, order=pop_stocks, orient='h', ax=ax[1])
ax[1].set_title('Popular Stocks');

In [ ]:
vola_df = pd.DataFrame(ror.groupby('symbol')['return'].std())

fig, ax = plt.subplots(1,2,sharey=True,figsize=(16,5))
plt.suptitle('Standard Deviation of daily return')
sns.barplot(x=vola_df.index, y=vola_df['return'], ax=ax[0], order=high_vola)
ax[0].set_title('Volatile Stocks')
ax[0].set_ylabel('Standard Deviation')
sns.barplot(x=vola_df.index, y=vola_df['return'], ax=ax[1], order=pop_stocks)
ax[1].set_title('Popular Stocks')
ax[1].set_ylabel('Standard Deviation');

### Winner Stocks and Loser Stocks 

In [ ]:
price_df = stocks.pivot(index='date',columns='symbol',values='close').dropna(how='any',axis=1)

for i in tqdm(range(price_df.shape[1])):
    for j in range(len(price_df)-1, -1, -1):
        price_df.iat[j,i] = price_df.iat[j,i]/price_df.iat[0,i]

In [ ]:
price_df

In [ ]:
winners = price_df.T.sort_values('2017-12-29', ascending=False)[:5].index
losers = price_df.T.sort_values('2017-12-29')[:5].index

fig,ax = plt.subplots(2,sharex=True,figsize=(14,8))
price_df[winners].plot(ax=ax[0])
ax[0].set_yscale('log')
price_df[losers].plot(ax=ax[1])
ax[1].set_yscale('log');

DISCA and DISCK are rather less-volatiles stocks, but they once expeienced sharp price decline. There might be stock split.